In [1]:
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from functools import reduce
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Create combined raw 5man lineup dataset, clean and save

In [23]:
n_man_lineup = 5
teams = [
        'ATL',
        'BRK',  # switches to BRK
        'BOS',
        'CHO',  # switches to CHA then CHO
        'CHI',
        'CLE',
        'DAL',
        'DEN',
        'DET',
        'GSW',
        'HOU',
        'IND',
        'LAC',
        'LAL',
        'MEM',  # switches to MEM
        'MIA',
        'MIL',
        'MIN',
        'NOP',  # switches to NOP
        'NYK',
        'OKC',  # switches to OKC
        'ORL',
        'PHI',
        'PHO',
        'POR',
        'SAC',
        'SAS',
        'TOR',
        'UTA',
        'WAS', # switches to WAS
        ]

dfs = []
for team in teams:    
    filename = f'bballref_data/lineups/{team}/{team}_{n_man_lineup}_man_lineups_1997_2020.csv'
    dfs.append(pd.read_csv(filename, index_col=0))
df = pd.concat(dfs)

df.to_csv(f'bballref_data/raw_all_teams_{n_man_lineup}_man_lineups_1997_2020.csv')

In [17]:
df.isna().sum().sort_values(ascending=False)

diff_fg3_pct      10
team               0
diff_orb           0
diff_pts           0
diff_fg            0
diff_fga           0
diff_fg_pct        0
diff_fg3           0
diff_fg3a          0
diff_efg_pct       0
diff_ft            0
diff_fta           0
diff_ft_pct        0
diff_orb_pct       0
year               0
diff_drb           0
diff_drb_pct       0
diff_trb           0
diff_trb_pct       0
diff_ast           0
diff_stl           0
diff_blk           0
diff_tov           0
diff_pf            0
player_ids         0
number_players     0
mp                 0
dtype: int64

In [18]:
for n in range(0, 550, 50):
    print(n, '  :  ', len(df[df['mp']>=n]))

0   :   14240
50   :   8428
100   :   3428
150   :   1964
200   :   1258
250   :   894
300   :   674
350   :   519
400   :   406
450   :   320
500   :   252


In [21]:
df_150min = df[df['mp']>=150].copy()
df_150min.shape

(1964, 27)

In [22]:
df_150min.isna().sum().sort_values(ascending=False)

team              0
diff_orb          0
diff_pts          0
diff_fg           0
diff_fga          0
diff_fg_pct       0
diff_fg3          0
diff_fg3a         0
diff_fg3_pct      0
diff_efg_pct      0
diff_ft           0
diff_fta          0
diff_ft_pct       0
diff_orb_pct      0
year              0
diff_drb          0
diff_drb_pct      0
diff_trb          0
diff_trb_pct      0
diff_ast          0
diff_stl          0
diff_blk          0
diff_tov          0
diff_pf           0
player_ids        0
number_players    0
mp                0
dtype: int64

In [24]:
df_150min.to_csv('datasets/lineups/5man_lineups_150min_1997_2020.csv')

## Clean 4 man

In [6]:
df_4man = pd.read_csv('bballref_data/raw_all_teams_4_man_lineups_1997_2020.csv', index_col=0)
df_4man.shape
df_4man.head()

(14240, 27)

,mp,diff_pts,diff_fg,diff_fga,diff_fg_pct,diff_fg3,diff_fg3a,diff_fg3_pct,diff_efg_pct,diff_ft,...,diff_trb_pct,diff_ast,diff_stl,diff_blk,diff_tov,diff_pf,player_ids,number_players,year,team
lineup,,,,,,,,,,,,,,,,,,,,,
M. Blaylock | C. Laettner | D. Mutombo | S. Smith ATL 1997,1768.966667,12.9,2.1,-3.2,0.040,1.8,5.3,-0.002,0.051,6.9,...,1.4,-0.1,2.8,2.6,-1.5,-1.3,"blaylmo01, laettch01, mutomdi01, smithst01",4,1997,ATL
M. Blaylock | T. Corbin | C. Laettner | D. Mutombo ATL 1997,1418.516667,15.1,3.3,-2.2,0.048,2.1,4.2,0.026,0.061,6.5,...,3.1,1.8,3.5,2.2,-2.2,-1.0,"blaylmo01, corbity01, laettch01, mutomdi01",4,1997,ATL
M. Blaylock | T. Corbin | C. Laettner | S. Smith ATL 1997,1384.100000,16.4,4.3,-0.3,0.049,1.9,4.0,0.022,0.060,6.0,...,4.3,2.3,3.6,1.8,-2.9,-0.9,"blaylmo01, corbity01, laettch01, smithst01",4,1997,ATL
M. Blaylock | T. Corbin | D. Mutombo | S. Smith ATL 1997,1350.250000,13.8,3.1,-1.9,0.045,1.8,4.1,0.018,0.056,5.8,...,3.6,1.0,3.3,2.3,-1.9,-1.0,"blaylmo01, corbity01, mutomdi01, smithst01",4,1997,ATL
T. Corbin | C. Laettner | D. Mutombo | S. Smith ATL 1997,1231.483333,15.6,3.2,-3.0,0.051,1.7,3.7,0.018,0.062,7.4,...,4.0,1.3,3.0,2.3,-1.7,-0.9,"corbity01, laettch01, mutomdi01, smithst01",4,1997,ATL


In [10]:
df_4man.isna().sum().sort_values(ascending=False)

team              0
diff_orb          0
diff_pts          0
diff_fg           0
diff_fga          0
diff_fg_pct       0
diff_fg3          0
diff_fg3a         0
diff_fg3_pct      0
diff_efg_pct      0
diff_ft           0
diff_fta          0
diff_ft_pct       0
diff_orb_pct      0
year              0
diff_drb          0
diff_drb_pct      0
diff_trb          0
diff_trb_pct      0
diff_ast          0
diff_stl          0
diff_blk          0
diff_tov          0
diff_pf           0
player_ids        0
number_players    0
mp                0
dtype: int64

In [11]:
df_4man.dtypes

mp                float64
diff_pts          float64
diff_fg           float64
diff_fga          float64
diff_fg_pct       float64
diff_fg3          float64
diff_fg3a         float64
diff_fg3_pct      float64
diff_efg_pct      float64
diff_ft           float64
diff_fta          float64
diff_ft_pct       float64
diff_orb          float64
diff_orb_pct      float64
diff_drb          float64
diff_drb_pct      float64
diff_trb          float64
diff_trb_pct      float64
diff_ast          float64
diff_stl          float64
diff_blk          float64
diff_tov          float64
diff_pf           float64
player_ids         object
number_players      int64
year                int64
team               object
dtype: object

In [12]:
for n in range(0, 550, 50):
    print(n, '  :  ', len(df_4man[df_4man['mp'] > n]))

0   :   14240
50   :   14240
100   :   14205
150   :   13253
200   :   10592
250   :   7895
300   :   5977
350   :   4650
400   :   3713
450   :   3006
500   :   2469


In [25]:
df_4man_300min = df_4man[df_4man['mp']>=300]
df_4man_300min.shape

(5977, 27)

In [28]:
df_4man_300min.to_csv('datasets/lineups/4man_lineups_300min_1997_2020.csv')

In [27]:
df_4man_300min.sort_values(by='diff_pts')

,mp,diff_pts,diff_fg,diff_fga,diff_fg_pct,diff_fg3,diff_fg3a,diff_fg3_pct,diff_efg_pct,diff_ft,...,diff_trb_pct,diff_ast,diff_stl,diff_blk,diff_tov,diff_pf,player_ids,number_players,year,team
lineup,,,,,,,,,,,,,,,,,,,,,
K. Bryant | R. Hibbert | J. Randle | D. Russell LAL 2016,652.300000,-24.9,-11.1,-4.5,-0.101,-3.1,-5.1,-0.050,-0.116,0.4,...,-11.4,-11.5,-1.4,0.0,3.0,0.5,"bryanko01, hibbero01, randlju01, russeda01",4,2016,LAL
E. Brand | R. Brown | H. Hawkins | D. Simpkins CHI 2000,330.233333,-24.8,-11.7,-12.0,-0.077,-0.7,-4.7,0.082,-0.079,-0.6,...,-4.7,-6.0,-6.8,-5.1,8.2,-0.4,"brandel01, brownra02, hawkihe01, simpkdi01",4,2000,CHI
D. Anderson | T. Nesby | M. Olowokandi | M. Taylor LAC 2000,357.483333,-24.2,-10.5,-1.0,-0.113,-2.9,-1.5,-0.180,-0.129,-0.3,...,-8.2,-9.1,-0.6,1.2,2.0,2.2,"anderde01, nesbyty01, olowomi01, tayloma01",4,2000,LAC
M. World Peace | E. Brand | R. Brown | H. Hawkins CHI 2000,344.366667,-23.9,-10.0,-6.1,-0.087,-1.5,-2.4,-0.069,-0.094,-2.3,...,-3.3,-6.5,-3.5,-2.8,5.2,2.4,"artesro01, brandel01, brownra02, hawkihe01",4,2000,CHI
K. Bryant | J. Clarkson | R. Hibbert | J. Randle LAL 2016,800.450000,-23.8,-10.8,-5.4,-0.093,-2.6,-4.7,-0.038,-0.104,0.3,...,-9.7,-10.7,-2.0,-0.5,3.7,0.8,"bryanko01, clarkjo01, hibbero01, randlju01",4,2016,LAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R. Alston | C. Hayes | T. McGrady | Y. Ming HOU 2007,542.916667,27.1,12.4,5.1,0.114,4.5,7.0,0.118,0.137,-2.1,...,13.6,11.5,1.0,1.0,-1.3,2.4,"alstora01, hayesch01, mcgratr01, mingya01",4,2007,HOU
M. Blaylock | G. Long | D. Mutombo | S. Smith ATL 1999,354.950000,28.2,7.2,-2.6,0.097,2.3,3.6,0.081,0.111,11.4,...,10.7,1.4,-0.2,3.3,-2.0,-3.0,"blaylmo01, longgr01, mutomdi01, smithst01",4,1999,ATL
M. Jordan | T. Kukoč | S. Pippen | D. Rodman CHI 1997,390.233333,29.1,11.2,8.7,0.078,2.1,1.2,0.087,0.087,4.5,...,9.8,11.2,4.9,1.5,-6.8,-1.9,"jordami01, kukocto01, pippesc01, rodmade01",4,1997,CHI
